# Lab | Hypothesis Testing

**Objective**

Welcome to the Hypothesis Testing Lab, where we embark on an enlightening journey through the realm of statistical decision-making! In this laboratory, we delve into various scenarios, applying the powerful tools of hypothesis testing to scrutinize and interpret data.

From testing the mean of a single sample (One Sample T-Test), to investigating differences between independent groups (Two Sample T-Test), and exploring relationships within dependent samples (Paired Sample T-Test), our exploration knows no bounds. Furthermore, we'll venture into the realm of Analysis of Variance (ANOVA), unraveling the complexities of comparing means across multiple groups.

So, grab your statistical tools, prepare your hypotheses, and let's embark on this fascinating journey of exploration and discovery in the world of hypothesis testing!

**Challenge 1**

In this challenge, we will be working with pokemon data. The data can be found here:

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv

In [1]:
#libraries
import pandas as pd
import scipy.stats as st
import numpy as np



In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv")
df

- We posit that Pokemons of type Dragon have, on average, more HP stats than Grass. Choose the propper test and, with 5% significance, comment your findings.

In [ ]:
#code here
df_dragon = df[df["Type 1"] == "Dragon"]["HP"]
df_grass = df[df["Type 1"] == "Grass"]["HP"]
df_grass.mean(), df_dragon.mean()

In [ ]:
st.ttest_ind(df_dragon, df_grass, equal_var=False)

- We posit that Legendary Pokemons have different stats (HP, Attack, Defense, Sp.Atk, Sp.Def, Speed) when comparing with Non-Legendary. Choose the propper test and, with 5% significance, comment your findings.


In [ ]:
#code here
df_legendary = df[df["Legendary"] == True]
df_not_legendary = df[df["Legendary"] == False]
df_legendary.mean(), df_not_legendary.mean()

In [ ]:
from statsmodels.multivariate.manova import MANOVA

In [ ]:
df.rename(columns={"Sp. Atk": "Sp_Atk", "Sp. Def": "Sp_Def"}, inplace=True)

In [ ]:
manova = MANOVA.from_formula("Legendary ~ HP + Attack + Defense + Sp_Atk + Sp_Def + Speed", df)
resultados = manova.mv_test()

In [ ]:
print(resultados)

**Challenge 2**

In this challenge, we will be working with california-housing data. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


**We posit that houses close to either a school or a hospital are more expensive.**

- School coordinates (-118, 37)
- Hospital coordinates (-122, 34)

We consider a house (neighborhood) to be close to a school or hospital if the distance is lower than 0.50.

Hint:
- Write a function to calculate euclidean distance from each house (neighborhood) to the school and to the hospital.
- Divide your dataset into houses close and far from either a hospital or school.
- Choose the propper test and, with 5% significance, comment your findings.
 

In [3]:
import math

In [4]:
school_coordinates = (-118, 37)
hospital_coordinates = (-122, 34)

In [ ]:
def euclidean_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def is_house_close(house_coordinates):
    distance_to_school = euclidean_distance(house_coordinates, school_coordinates)
    distance_to_hospital = euclidean_distance(house_coordinates, hospital_coordinates)
    
    if distance_to_school < 0.50 or distance_to_hospital < 0.50:
        return True
    else:
        return False

In [9]:
map(is_house_close, df[["longitude", "latitude"]].values)

In [10]:
print(list(map(is_house_close, df[["longitude", "latitude"]].values)))

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

In [11]:
df["is_close"] = list(map(is_house_close, df[["longitude", "latitude"]].values))

In [12]:
df_close = df[df["is_close"] == True]
df_far = df[df["is_close"] == False]

In [ ]:
print("Descriptive Statistics for Close Houses:")
print(df_close['median_house_value'].describe())

print("\nDescriptive Statistics for Far Houses:")
print(df_far['median_house_value'].describe())

shapiro_close = st.shapiro(df_close['median_house_value'])
shapiro_far = st.shapiro(df_far['median_house_value'])

print("\nShapiro-Wilk Test for Normality (Close Houses):", shapiro_close)
print("Shapiro-Wilk Test for Normality (Far Houses):", shapiro_far)

if shapiro_close.pvalue > 0.05 and shapiro_far.pvalue > 0.05:
    t_statistic, p_value = st.ttest_ind(df_close['median_house_value'], df_far['median_house_value'])
    test_type = "Independent t-test"
else:
    u_statistic, p_value = st.mannwhitneyu(df_close['median_house_value'], df_far['median_house_value'])
    test_type = "Mann-Whitney U test"

print(f"\n{test_type} Results:")
print("P-value:", p_value)

if p_value < 0.05:
    print("\nThere is a significant difference in house prices between houses close to a school/hospital and those far away.")
else:
    print("\nThere is no significant difference in house prices between houses close to a school/hospital and those far away.")

Descriptive Statistics for Close Houses:
count         5.000000
mean      92460.000000
std       14823.730974
min       74200.000000
25%       80600.000000
50%       94500.000000
75%      104700.000000
max      108300.000000
Name: median_house_value, dtype: float64

Descriptive Statistics for Far Houses:
count     16995.000000
mean     207334.699029
std      115983.871088
min       14999.000000
25%      119500.000000
50%      180400.000000
75%      265000.000000
max      500001.000000
Name: median_house_value, dtype: float64

Shapiro-Wilk Test for Normality (Close Houses): ShapiroResult(statistic=0.9228575371279246, pvalue=0.5485392407287775)
Shapiro-Wilk Test for Normality (Far Houses): ShapiroResult(statistic=0.9120818664848804, pvalue=1.398333360821156e-70)

Mann-Whitney U test Results:
P-value: 0.005835992784005986

There is a significant difference in house prices between houses close to a school/hospital and those far away.


c:\Users\javie\anaconda3\lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 16995.
  res = hypotest_fun_out(*samples, **kwds)
